# London Fire Incidents Dimensionality Reduction

In [1]:
%matplotlib inline 
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### Downloading and loadng the data

In [2]:
# Loading the data
# import data_download

data_dir = "LFB-data"
# LFB_data = pd.read_csv(os.path.join(data_dir, "LFB Incident data - Datastore - with notional cost and UPRN from January 2009.csv"))
LFB_data = pd.read_csv(os.path.join(data_dir, "lfb_incident.csv"))

# Total memory used
print(f'Total Memory Used : {round(LFB_data.memory_usage(deep=True).sum()/(1024*1024), 2)} MB')
LFB_data.head()

Total Memory Used : 2091.97 MB


,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,...,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpHoursRoundUp,Notional Cost (£),NumCalls
0,235138081,01 Jan 2009,2009,00:00:37,0,Special Service,Special Service,RTC,Road Vehicle,Car,...,319.0,Battersea,342.0,Clapham,2.0,2.0,2.0,1.0,255.0,1.0
1,1091,01 Jan 2009,2009,00:00:46,0,Special Service,Special Service,Assist other agencies,Outdoor,Lake/pond/reservoir,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,2091,01 Jan 2009,2009,00:03:00,0,Fire,Secondary Fire,NaN,Outdoor,Road surface/pavement,...,308.0,Edmonton,NaN,NaN,1.0,1.0,1.0,1.0,255.0,2.0
3,3091,01 Jan 2009,2009,00:04:27,0,Fire,Secondary Fire,NaN,Outdoor,Domestic garden (vegetation not equipment),...,210.0,Hillingdon,NaN,NaN,1.0,1.0,1.0,1.0,255.0,2.0
4,5091,01 Jan 2009,2009,00:05:39,0,Fire,Secondary Fire,NaN,Outdoor,Cycle path/public footpath/bridleway,...,233.0,Holloway,250.0,Holloway,1.0,2.0,2.0,1.0,255.0,1.0


### Primary Data analysis

In [3]:
LFB_data.shape

(1465060, 39)

In [4]:
LFB_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465060 entries, 0 to 1465059
Data columns (total 39 columns):
 #   Column                                  Non-Null Count    Dtype  
---  ------                                  --------------    -----  
 0   IncidentNumber                          1465060 non-null  object 
 1   DateOfCall                              1465060 non-null  object 
 2   CalYear                                 1465060 non-null  int64  
 3   TimeOfCall                              1465060 non-null  object 
 4   HourOfCall                              1465060 non-null  int64  
 5   IncidentGroup                           1465060 non-null  object 
 6   StopCodeDescription                     1465060 non-null  object 
 7   SpecialServiceType                      459204 non-null   object 
 8   PropertyCategory                        1465060 non-null  object 
 9   PropertyType                            1465060 non-null  object 
 10  AddressQualifier              

We can see that there are missing values

## Data Preprocessing

#### Time Processing

In [5]:
# We need to drop the existing Hour of call and create our own
LFB_data.drop('HourOfCall', axis=1, inplace=True)

# Create a new column from the DateOfCall column.
LFB_data['YearOfCall'], LFB_data['MonthOfCall'] = LFB_data['DateOfCall'].apply(lambda x: x.split(" ")[2]),LFB_data['DateOfCall'].apply(lambda x: x.split(" ")[1])
LFB_data['HourOfCall'] = LFB_data['TimeOfCall'].apply(lambda x: x.split(":")[0])

# Dropping unnecessary time columns
LFB_data.drop(['IncidentNumber','TimeOfCall','DateOfCall','CalYear'], axis=1, inplace = True)

#### Service and group processing

In [6]:
LFB_data.drop(['StopCodeDescription','SpecialServiceType'], axis=1, inplace = True)
LFB_data.sample(4)

,IncidentGroup,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,Postcode_district,UPRN,USRN,IncGeo_BoroughCode,IncGeo_BoroughName,...,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpHoursRoundUp,Notional Cost (£),NumCalls,YearOfCall,MonthOfCall,HourOfCall
274122,Special Service,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,Within same building,NaN,N1,0.0,20900805.0,E09000012,HACKNEY,...,NaN,1.0,1.0,1.0,1.0,260.0,1.0,2011,Feb,12
182757,False Alarm,Outdoor,Domestic garden (vegetation not equipment),In street close to gazetteer location,SE25 4BT,SE25,NaN,NaN,E09000008,CROYDON,...,NaN,1.0,1.0,1.0,1.0,260.0,1.0,2010,May,21
313482,False Alarm,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,Within same building,NaN,CR4,0.0,22103995.0,E09000024,MERTON,...,Tooting,2.0,3.0,3.0,1.0,260.0,1.0,2011,Jun,18
188330,Special Service,Road Vehicle,Multiple Vehicles,In street outside gazetteer location,N10 1AA,N10,NaN,NaN,E09000003,BARNET,...,Hornsey,1.0,2.0,2.0,1.0,260.0,1.0,2010,Jun,18


#### Inc Geo data

In [7]:
# Inc Geo borough name and code all describe the same borough. 

# Number of distinct rows for each column
print(LFB_data[['IncGeo_BoroughCode', 'IncGeo_BoroughName']].nunique())

IncGeo_BoroughCode    33
IncGeo_BoroughName    33
dtype: int64


In [8]:
# we can join the two columns into one to avoid to many vatiables after encoding
LFB_data['IncGeo_Borough'] = LFB_data['IncGeo_BoroughCode'] + '-' +LFB_data['IncGeo_BoroughName']

# Finally we need to drop the two columns
LFB_data.drop(['IncGeo_BoroughCode','IncGeo_BoroughName'], axis=1, inplace=True)
LFB_data['IncGeo_Borough'].tail(3)

1465057       E09000017-HILLINGDON
1465058    E09000030-TOWER HAMLETS
1465059           E09000007-CAMDEN
Name: IncGeo_Borough, dtype: object

In [9]:
# Similary for IncGeo code, ward name and ward name new, all describe the same ward

# Number of distinct rows for each column
print(LFB_data[['IncGeo_WardCode', 'IncGeo_WardName', 'IncGeo_WardNameNew']].nunique())
LFB_data['IncGeo_Ward'] = LFB_data['IncGeo_WardCode'] + '-' +LFB_data['IncGeo_WardName']

# Drop Unecessary Columns
LFB_data.drop(['IncGeo_WardCode','IncGeo_WardName','IncGeo_WardNameNew'], axis=1, inplace=True)
LFB_data['IncGeo_Ward'].sample(3)

IncGeo_WardCode       1274
IncGeo_WardName       1560
IncGeo_WardNameNew    1546
dtype: int64


1420527     E05000123-PENGE AND CATOR
965624     E05011096-Camberwell Green
939185             E05000433-THORNTON
Name: IncGeo_Ward, dtype: object

#### Slicing Northing_m and easting_m to 4 last digits

In [10]:
# process Northing and easting data and process it ito categorical data using pandas cut

LFB_data['Easting_rounded']= LFB_data['Easting_rounded'].apply(lambda x: int(str(x)[2:]))
LFB_data['Easting_rounded']= pd.cut(LFB_data['Easting_rounded'], bins=10, labels=['Easting_rounded_0','Easting_rounded_1','Easting_rounded_2','Easting_rounded_3','Easting_rounded_4','Easting_rounded_5','Easting_rounded_6','Easting_rounded_7','Easting_rounded_8','Easting_rounded_9'])

LFB_data['Northing_rounded']= LFB_data['Northing_rounded'].apply(lambda x: int(str(x)[2:]))
LFB_data['Northing_rounded']= pd.cut(LFB_data['Northing_rounded'], bins=10, labels=['Northing_rounded_0','Northing_rounded_1','Northing_rounded_2','Northing_rounded_3','Northing_rounded_4','Northing_rounded_5','Northing_rounded_6','Northing_rounded_7','Northing_rounded_8','Northing_rounded_9'])

In [11]:
LFB_data.isnull().sum()

IncidentGroup                                  0
PropertyCategory                               0
PropertyType                                   0
AddressQualifier                               0
Postcode_full                             706377
Postcode_district                              0
UPRN                                      141333
USRN                                      162871
ProperCase                                     0
Easting_m                                 706377
Northing_m                                706377
Easting_rounded                                0
Northing_rounded                               0
Latitude                                  706377
Longitude                                 706377
FRS                                            0
IncidentStationGround                          1
FirstPumpArriving_AttendanceTime          119752
FirstPumpArriving_DeployedFromStation     119764
SecondPumpArriving_AttendanceTime         939563
SecondPumpArriving_D

#### Dropping Unecessary columns

## Exploratory data analysis

### Numerical data

In [ ]:
# select numeric columns
df_numeric = LFB_data.select_dtypes(include=[np.number])

print("Numeric data shape : ",df_numeric.shape)
df_numeric.columns.values

In [ ]:
df_numeric.describe().T.apply(lambda s: s.apply('{0:.2f}'.format))

We can tell from the data above that there are outliers in the numeric data.
For instance, there are values that have a very huge diffrence between the 75th percentile and maximum value

#### Numerical data visualization

In [ ]:
fig = plt.figure(figsize= (20,50))

for i in range(len(df_numeric.columns)):
    column = df_numeric.columns[i]
    sub = fig.add_subplot(9,3, i+1)
    chart = sns.boxplot(data=df_numeric, y=column, x = LFB_data["IncidentGroup"])
    chart.set_title(column + " by incident group")

#### Missing values on Numeric data

In [ ]:
# Check for any missing values
print("Number of cols with Missing Vals: ",df_numeric.isna().any().sum())
display(df_numeric.isna().sum())

<h6>We need to fix the missing values to cluster around the mean value<br>We will consider randomizing the missing values between 30% and 70%</h6>

#### Missing values

In [ ]:

for i in df_numeric[['UPRN', 'USRN', 'Easting_m', 'Northing_m','Easting_rounded', 'Northing_rounded',
                    'FirstPumpArriving_AttendanceTime','SecondPumpArriving_AttendanceTime',
                    'NumStationsWithPumpsAttending', 'NumPumpsAttending', 'PumpCount',
                    'PumpHoursRoundUp', 'Notional Cost (£)', 'NumCalls']]:
    df_numeric.fillna(0, inplace=True)
    # Set 30 and 70th percentile and round off to 2
    rand_30_70 = random.uniform(round(np.percentile(df_numeric[i],30),2), round(np.percentile(df_numeric[i],70),2)) 
    for j in i:
        if j == 0:
            df_numeric.replace(to_replace=0, value=rand_30_70, inplace=True)
            

#### Outliers in numerical data

In [ ]:
for i in df_numeric.columns:
    df_numeric.fillna(df_numeric[i].mode()[0], inplace = True)
    highest_val = df_numeric[i].mean() + 3*df_numeric[i].std()
    lowest_val = df_numeric[i].mean() - 3*df_numeric[i].std()
    print(f"Range for {i} : ", round(lowest_val,2), " to ",round(highest_val,2))
    
#     Trimming the outliers
    df_numeric[i]= np.where(df_numeric[i]>highest_val, highest_val,
                           np.where(df_numeric[i]<lowest_val, lowest_val,
                                   df_numeric[i]))
#     (df_numeric[i]>=lowest_val)&(df_numeric[i]<=highest_val)

print( "\n","*"*120)
df_numeric.describe().T.apply(lambda s: s.apply('{0:.2f}'.format))

In [ ]:
df_numeric.isnull().sum()

### Categorical Data

In [ ]:
df_categorical = LFB_data.select_dtypes(exclude=[np.number])
print(df_categorical.shape)
print( "\n","-"*120)
df_categorical.columns.values

In [ ]:
df_categorical.sample(10)

In [ ]:
df_categorical.drop(['IncidentNumber','StopCodeDescription', 'SpecialServiceType', 'IncGeo_WardNameNew', 'SecondPumpArriving_DeployedFromStation'], axis = 1, inplace = True)

In [ ]:
# Drop the incidentnumber, postcode_full,  column since we really dont need it
df_categorical.drop(['Postcode_full'], axis = 1, inplace=True)

In [ ]:
# Show new dataframe

df_categorical

In [ ]:
# Show unique values for each categorcal variable

df_categorical.nunique()

### Joining dataframes

In [ ]:
final_df = pd.concat([df_numeric, df_categorical], axis = 1)
final_df.shape

In [ ]:
final_df.sample(7)

### One hot encoding

In [ ]:
final_df = pd.get_dummies(final_df)
final_df.tail()

# Dimensionality reduction

In [ ]:
random.seed(10)
rand_num=random.sample(range(len(final_df)), 10000)
rand_num[:10]

In [ ]:
working_df = final_df.iloc[rand_num]
working_df.sample(10)

In [ ]:
working_df.shape

## T-Stochastic Neighbor Embedding (t-SNE)

In [ ]:
%%time
import scipy
import time
from scipy.spatial.distance import pdist
from sklearn.manifold import TSNE
from scipy import stats
# A=scipy.spatial.distance.pdist(working_df, metric='euclidean')
# kendTSNE=[]

start_time = time.time()
tsne = TSNE(n_components = 2, verbose=1, learning_rate=200, n_iter=500)
tsne_result = tsne.fit_transform(X=working_df)
end_time = time.time()
print("Learning completed in {} seconds".format(end_time - start_time))

In [ ]:
tsne_df = pd.DataFrame({"t-SNE 1":tsne_result[:,0], "t-SNE 2":tsne_result[:,1]})
tsne_df.head()

In [ ]:
# Plotting the tsne data

fig, ax = plt.subplots(1, figsize=(15,10))
sns.scatterplot(x = 't-SNE 1', y= 't-SNE 2', data = tsne_df, ax = ax, s=20, palette = 'dark')
sns.color_palette("hls", 10)
lim = (tsne_df.min()-5, tsne_df.max()+5)
ax.set_title('t-SNE Visualization of Incident Group', fontsize = 16, weight = 'bold')
ax.legend(bbox_to_anchor = (1,1), loc =2, borderaxespad = 0.0)

## Same Degree Distribution

In [ ]:
from scipy import linalg
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from scipy.sparse import csr_matrix, issparse
from sklearn.neighbors import NearestNeighbors
from sklearn.base import BaseEstimator
from sklearn.utils import check_random_state
from sklearn.utils._openmp_helpers import _openmp_effective_n_threads
from sklearn.utils.validation import check_non_negative
from sklearn.utils.validation import _deprecate_positional_args
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import pairwise_distances

MACHINE_EPSILON = np.finfo(np.double).eps